# Michelin Rated Restaurants in France - Analysis

In [1]:
# Module import (comprehensive for entire notebook)
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import geopandas as gpd
import mapclassify
import folium
import branca

Importing the processed data

In [2]:
# All restaurants
all_france = pd.read_csv("../../data/France/all_restaurants.csv")

In [3]:
# We define Paris, 75
paris = all_france[all_france['city'] == 'Paris']

# Everything but Paris
rest_of_france = all_france[all_france['city'] != 'Paris']

----
&nbsp;
#### Michelin Rated Restaurants Information

In [4]:
def dataframe_info(data):
    print(f"Unique Departments: {data['department'].nunique()}")
    print(f"Unique Cities: {data['city'].nunique()}")
    print(f"Shape: {data.shape}")
    display(data.head(3))

In [5]:
dataframe_info(all_france)

Unique Departments: 92
Unique Cities: 617
Shape: (1033, 15)


,name,address,city,department_num,department,region,country,price,price_num,cuisine,url,award,stars,longitude,latitude
0,La Vague d'Or - Cheval Blanc St-Tropez,"Plage de la Bouillabaisse, Saint-Tropez, 83990...",Saint-Tropez,83,Var,Provence-Alpes-Côte d'Azur,France,€€€€,4,"Creative, Modern Cuisine",https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,3.0,6.626154,43.266585
1,René et Maxime Meilleur,"Hameau de Saint-Marcel, Saint-Martin-de-Belle...",Saint-Martin-de-Belleville,73,Savoie,Auvergne-Rhône-Alpes,France,€€€€,4,"Creative, Regional Cuisine",https://www.la-bouitte.com/fr/,3 MICHELIN Stars,3.0,6.513306,45.369046
2,Kei,"5 rue du Coq-Héron, Paris, 75001, France",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Creative",https://www.restaurant-kei.fr/,3 MICHELIN Stars,3.0,2.342285,48.864395


In [6]:
dataframe_info(paris)

Unique Departments: 1
Unique Cities: 1
Shape: (159, 15)


,name,address,city,department_num,department,region,country,price,price_num,cuisine,url,award,stars,longitude,latitude
2,Kei,"5 rue du Coq-Héron, Paris, 75001, France",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Creative",https://www.restaurant-kei.fr/,3 MICHELIN Stars,3.0,2.342285,48.864395
11,Le Pré Catelan,"Route de Suresnes - bois de Boulogne, Paris, 7...",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Classic Cuisine",https://www.leprecatelan.com/,3 MICHELIN Stars,3.0,2.250718,48.863937
13,Épicure,"Le Bristol, 112 rue du Faubourg-Saint-Honoré, ...",Paris,75,Paris,Île-de-France,France,€€€€,4,Modern Cuisine,https://www.oetkercollection.com/fr/hotels/le-...,3 MICHELIN Stars,3.0,2.314625,48.871762


----
&nbsp;
## Analysis of Michelin Stars by department and region

### `top_restaurants_by_department` function

Imported below and defined in `functions.py`, this function analyses a dataset of restaurants and outputs the top $n$ departments with the highest count of restaurants with a specified Michelin star rating.

Usage:
```python
top_restaurants_by_department(data, star_rating, top_n, display_restaurants=True)
```
----
&nbsp;
`data` is a pandas DataFrame that contains restaurant information. It expects the following columns: 'department', 'stars', 'name', 'city', and 'department_num'.

`star_rating` is an integer (1, 2, or 3) specifying the Michelin star rating of interest.

`top_n` is an integer specifying the number of departments to consider.

`display_restaurants` is an optional boolean flag (default: True), Prints the name and city.

`display_info` is an optional boolean flag (default: False). Prints style of cuisine and url

In [7]:
from functions import top_restaurants_by_department

Top 3 departments by three star restaurants

In [10]:
# excludes Paris
top_restaurants_by_department(rest_of_france, 3, 2, display_info=True)

Top 2 departments with most ⭐⭐⭐ restaurants:


Department: Bouches-du-Rhône (13)
4 ⭐⭐⭐ Restaurants

Restaurant: AM par Alexandre Mazzia
Location: Marseille
Style of Cuisine: Creative
URL: https://www.alexandre-mazzia.com/

Restaurant: Le Petit Nice
Location: Marseille
Style of Cuisine: Seafood
URL: https://www.passedat.fr

Restaurant: La Villa Madie
Location: Cassis
Style of Cuisine: Creative, Mediterranean Cuisine
URL: https://lavillamadie.com/

Restaurant: L'Oustau de Baumanière
Location: Les Baux-de-Provence
Style of Cuisine: Creative
URL: https://www.baumaniere.com/



Department: Haute-Savoie (74)
2 ⭐⭐⭐ Restaurants

Restaurant: Flocons de Sel
Location: Megève
Style of Cuisine: Modern Cuisine
URL: https://www.floconsdesel.com/fr/

Restaurant: Le Clos des Sens
Location: Annecy
Style of Cuisine: Creative
URL: https://www.closdessens.com/





In [11]:
# We specify 'top5' however, the paris dataset has 1 department; 75
top_restaurants_by_department(paris, 3, 5, display_info=True)

⭐⭐⭐ Restaurants in Paris

Department: Paris (75)
9 ⭐⭐⭐ Restaurants

Restaurant: Kei
Location: Paris
Style of Cuisine: Modern Cuisine, Creative
URL: https://www.restaurant-kei.fr/

Restaurant: Le Pré Catelan
Location: Paris
Style of Cuisine: Modern Cuisine, Classic Cuisine
URL: https://www.leprecatelan.com/

Restaurant: Épicure
Location: Paris
Style of Cuisine: Modern Cuisine
URL: https://www.oetkercollection.com/fr/hotels/le-bristol-paris/restaurants-et-bar/epicure/

Restaurant: Arpège
Location: Paris
Style of Cuisine: Creative
URL: https://www.alain-passard.com/

Restaurant: L'Ambroisie
Location: Paris
Style of Cuisine: Classic Cuisine
URL: https://www.ambroisie-paris.com/

Restaurant: Le Cinq
Location: Paris
Style of Cuisine: Modern Cuisine
URL: https://www.fourseasons.com

Restaurant: Plénitude - Cheval Blanc Paris
Location: Paris
Style of Cuisine: Creative
URL: https://www.chevalblanc.com/fr/maison/paris/

Restaurant: Pierre Gagnaire
Location: Paris
Style of Cuisine: Creative
URL: 

### `top_restaurants_by_region` function

Again, defined in `functions.py`, it analyses a dataset of restaurants and outputs the top $n$ departments with the highest count of restaurants with a specified Michelin star rating. This iteration allows the argument 'all' for `star_rating`

In [12]:
from functions import top_restaurants_by_region

In [13]:
# For brevity, the display_restaurants flag is set False
# Using 'all' parameter
top_restaurants_by_region(rest_of_france, 'all', 5, display_restaurants=False)

Top 5 regions with most starred restaurants:


Region: Auvergne-Rhône-Alpes
95 Starred Restaurants

8 ⭐⭐⭐ Restaurants

15 ⭐⭐ Restaurants

72 ⭐ Restaurants



Region: Provence-Alpes-Côte d'Azur
77 Starred Restaurants

6 ⭐⭐⭐ Restaurants

5 ⭐⭐ Restaurants

66 ⭐ Restaurants



Region: Nouvelle-Aquitaine
57 Starred Restaurants

1 ⭐⭐⭐ Restaurant

7 ⭐⭐ Restaurants

49 ⭐ Restaurants



Region: Grand Est
51 Starred Restaurants

1 ⭐⭐⭐ Restaurant

9 ⭐⭐ Restaurants

41 ⭐ Restaurants



Region: Occitanie
49 Starred Restaurants

1 ⭐⭐⭐ Restaurant

8 ⭐⭐ Restaurants

40 ⭐ Restaurants





In [14]:
top_restaurants_by_region(all_france, 'all', 'all', display_restaurants=False)

Regions with starred restaurants:


Region: Île-de-France
134 Starred Restaurants

9 ⭐⭐⭐ Restaurants

15 ⭐⭐ Restaurants

110 ⭐ Restaurants



Region: Auvergne-Rhône-Alpes
95 Starred Restaurants

8 ⭐⭐⭐ Restaurants

15 ⭐⭐ Restaurants

72 ⭐ Restaurants



Region: Provence-Alpes-Côte d'Azur
77 Starred Restaurants

6 ⭐⭐⭐ Restaurants

5 ⭐⭐ Restaurants

66 ⭐ Restaurants



Region: Nouvelle-Aquitaine
57 Starred Restaurants

1 ⭐⭐⭐ Restaurant

7 ⭐⭐ Restaurants

49 ⭐ Restaurants



Region: Grand Est
51 Starred Restaurants

1 ⭐⭐⭐ Restaurant

9 ⭐⭐ Restaurants

41 ⭐ Restaurants



Region: Occitanie
49 Starred Restaurants

1 ⭐⭐⭐ Restaurant

8 ⭐⭐ Restaurants

40 ⭐ Restaurants



Region: Brittany
37 Starred Restaurants

2 ⭐⭐ Restaurants

35 ⭐ Restaurants



Region: Bourgogne-Franche-Comté
35 Starred Restaurants

1 ⭐⭐⭐ Restaurant

6 ⭐⭐ Restaurants

28 ⭐ Restaurants



Region: Pays de la Loire
23 Starred Restaurants

1 ⭐⭐⭐ Restaurant

1 ⭐⭐ Restaurant

21 ⭐ Restaurants



Region: Centre-Val de Loire
21 St